In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the Oxford-IIIT Pet Dataset
dataset_name = 'oxford_iiit_pet'
dataset, info = tfds.load(dataset_name, split='train', shuffle_files=True, with_info=True)

# Preprocess the dataset for object detection
def preprocess_data(example):
    image = tf.image.resize(example['image'], (300, 300)) / 255.0
    bbox = example['objects']['bbox']
    label = example['objects']['label']
    return image, {'bbox': bbox, 'label': label}

dataset = dataset.map(preprocess_data).batch(32)

# Load SSD MobileNet v2 model
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Fine-tune the model
base_model = detector.signatures['default']
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.Huber()

# Training loop
def train_model(dataset, epochs=5):
    for epoch in range(epochs):
        epoch_loss = 0
        for images, labels in dataset:
            with tf.GradientTape() as tape:
                predictions = base_model(images)
                loss = loss_fn(labels['bbox'], predictions['detection_boxes'])
            gradients = tape.gradient(loss, base_model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, base_model.trainable_variables))
            epoch_loss += loss.numpy()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss}")

train_model(dataset)

# Function to display results
def display_results(image, boxes):
    plt.imshow(image)
    ax = plt.gca()
    for box in boxes:
        ymin, xmin, ymax, xmax = box
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

# Test the model on a sample image
sample = next(iter(dataset))
predictions = base_model(sample[0])
display_results(sample[0][0].numpy(), predictions['detection_boxes'][0].numpy())


KeyError: in user code:

    File "<ipython-input-1-168e319ff487>", line 15, in preprocess_data  *
        bbox = example['objects']['bbox']

    KeyError: 'objects'


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the Oxford-IIIT Pet Dataset
dataset_name = 'oxford_iiit_pet'
dataset, info = tfds.load(dataset_name, split='train', shuffle_files=True, with_info=True)

# Preprocess the dataset for object detection
def preprocess_data(example):
    image = tf.image.resize(example['image'], (300, 300)) / 255.0
    # Access bounding box and label information using the correct keys
    bbox = example['bbox']  # Changed from example['objects']['bbox']
    label = example['label']  # Changed from example['objects']['label']
    return image, {'bbox': bbox, 'label': label}

dataset = dataset.map(preprocess_data).batch(32)

# Load SSD MobileNet v2 model
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Fine-tune the model
base_model = detector.signatures['default']
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.Huber()

# Training loop
def train_model(dataset, epochs=5):
    for epoch in range(epochs):
        epoch_loss = 0
        for images, labels in dataset:
            with tf.GradientTape() as tape:
                predictions = base_model(images)
                loss = loss_fn(labels['bbox'], predictions['detection_boxes'])
            gradients = tape.gradient(loss, base_model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, base_model.trainable_variables))
            epoch_loss += loss.numpy()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss}")

train_model(dataset)

# Function to display results
def display_results(image, boxes):
    plt.imshow(image)
    ax = plt.gca()
    for box in boxes:
        ymin, xmin, ymax, xmax = box
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

# Test the model on a sample image
sample = next(iter(dataset))
predictions = base_model(sample[0])
display_results(sample[0][0].numpy(), predictions['detection_boxes'][0].numpy())

KeyError: in user code:

    File "<ipython-input-2-9e4d47148577>", line 16, in preprocess_data  *
        bbox = example['bbox']  # Changed from example['objects']['bbox']

    KeyError: 'bbox'


In [ ]:
def preprocess_data(example):
    image = tf.image.resize(example['image'], (300, 300)) / 255.0
    # Access bounding box and label information using the correct keys
    bbox = example['objects']['bbox']  # Access bbox from within 'objects'
    label = example['objects']['label']  # Access label from within 'objects'
    return image, {'bbox': bbox, 'label': label}